<a href="https://colab.research.google.com/github/DeepsphereAI/DeepsphereProjects/blob/main/AutoQuestGen/matchthefollowing_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing from https://github.com/boudinfl/pke library for Python Keyword extraction
# We use a fixed commit as the later changes might break the code. If it was on pip we would have used exact version number.

!pip install --quiet git+https://github.com/boudinfl/pke.git@dc4d5f21e0ffe64c4df93c46146d29d1c522476b #python keyword extraction library
!pip install --quiet flashtext==2.7 #This module can be used to replace keywords in sentences or extract keywords from sentences. It is based on the 
# FlashText algorithm.
!pip install --quiet transformers==2.9.0
!pip install --quiet nltk==3.4.5

     |████████████████████████████████| 245kB 8.7MB/s 
     |████████████████████████████████| 645kB 7.5MB/s 
     |████████████████████████████████| 1.2MB 15.5MB/s 
     |████████████████████████████████| 870kB 43.9MB/s 
     |████████████████████████████████| 5.6MB 17.2MB/s 
     |████████████████████████████████| 1.5MB 9.0MB/s 


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
import json
import requests
import string
import re
import nltk
import string
import itertools
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt') #Punkt Sentence Tokenizer. This tokenizer divides a text into a list of sentences, by using an unsupervised algorithm 
# to build a model for abbreviation words, collocations, and words that start sentences
import pke
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import traceback
#Traceback is a python module that provides a standard interface to extract, format and print stack traces of a python program. When it prints 
# the stack trace it exactly mimics the behaviour of a python interpreter.
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor

def tokenize_sentences(text):
    sentences = sent_tokenize(text)
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
import textwrap
# The textwrap module can be used for wrapping and formatting of plain text. This module 
# provides formatting of text by adjusting the line breaks in the input paragraph.

text = """
“The science and engineering of making intelligent machines, especially intelligent computer programs”. -John McCarthy-

John McCarthy
Artificial Intelligence is an approach to make a computer, a robot, or a product to think how smart human think. AI is a study of how human brain think, learn, decide and work, when it tries to solve problems. And finally this study outputs intelligent software systems.The aim of AI is to improve computer functions which are related to human knowledge, for example, reasoning, learning, and problem-solving.
The intelligence is intangible. It is composed of
Reasoning
Learning
Problem Solving
Perception
Linguistic Intelligence
The objectives of AI research are reasoning, knowledge representation, planning, learning, natural language processing, realization, and ability to move and manipulate objects. There are long-term goals in the general intelligence sector.

Approaches include statistical methods, computational intelligence, and traditional coding AI. During the AI research related to search and mathematical optimization, artificial neural networks and methods based on statistics, probability, and economics, we use many tools. Computer science attracts AI in the field of science, mathematics, psychology, linguistics, philosophy and so on.
Trending AI Articles:
1. Cheat Sheets for AI, Neural Networks, Machine Learning, Deep Learning & Big Data
2. Data Science Simplified Part 1: Principles and Process
3. Getting Started with Building Realtime API Infrastructure
4. AI & NLP Workshop
Applications of AI
· Gaming − AI plays important role for machine to think of large number of possible positions based on deep knowledge in strategic games. for example, chess,river crossing, N-queens problems and etc.
· Natural Language Processing − Interact with the computer that understands natural language spoken by humans.
· Expert Systems − Machine or software provide explanation and advice to the users.
· Vision Systems − Systems understand, explain, and describe visual input on the computer.
· Speech Recognition − There are some AI based speech recognition systems have ability to hear and express as sentences and understand their meanings while a person talks to it. For example Siri and Google assistant.
· Handwriting Recognition − The handwriting recognition software reads the text written on paper and recognize the shapes of the letters and convert it into editable text.
· Intelligent Robots − Robots are able to perform the instructions given by a human.
Major Goals
Knowledge reasoning
Planning
Machine Learning
Natural Language Processing
Computer Vision
Robotics
IBM Watson

“Watson” is an IBM supercomputer that combines Artificial Intelligence (AI) and complex inquisitive programming for ideal execution as a “question answering” machine. The supercomputer is named for IBM’s founder, Thomas J. Watson.
IBM Watson is at the forefront of the new era of computing. At the point when IBM Watson made, IBM communicated that “more than 100 particular techniques are used to inspect perceive sources, find and make theories, find and score affirm, and combination and rank speculations.” recently, the Watson limits have been expanded and the way by which Watson works has been changed to abuse new sending models (Watson on IBM Cloud) and propelled machine learning capacities and upgraded hardware open to architects and authorities. It isn’t any longer completely a request answering figuring system arranged from Q&A joins yet can now ‘see’, ‘hear’, ‘read’, ‘talk’, ‘taste’, ‘translate’, ‘learn’ and ‘endorse’.
"""
wrapper = textwrap.TextWrapper(width=150)
word_list = wrapper.wrap(text=text)
for element in word_list: 
  print(element) 

 “The science and engineering of making intelligent machines, especially intelligent computer programs”. -John McCarthy-  John McCarthy Artificial
Intelligence is an approach to make a computer, a robot, or a product to think how smart human think. AI is a study of how human brain think, learn,
decide and work, when it tries to solve problems. And finally this study outputs intelligent software systems.The aim of AI is to improve computer
functions which are related to human knowledge, for example, reasoning, learning, and problem-solving. The intelligence is intangible. It is composed
of Reasoning Learning Problem Solving Perception Linguistic Intelligence The objectives of AI research are reasoning, knowledge representation,
planning, learning, natural language processing, realization, and ability to move and manipulate objects. There are long-term goals in the general
intelligence sector.  Approaches include statistical methods, computational intelligence, and traditional coding AI.

In [9]:
def get_keywords(text):
    out=[]
    try:
        extractor = pke.unsupervised.YAKE()
        extractor.load_document(input=text)
        # pos = {'VERB', 'ADJ', 'NOUN'}
        pos ={'NOUN'}
        stoplist = list(string.punctuation)
        stoplist += ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
        stoplist += stopwords.words('english')
        extractor.candidate_selection(n=2,pos=pos, stoplist=stoplist)

        extractor.candidate_weighting(window=3,
                                      stoplist=stoplist,
                                      use_stems=False)

        keyphrases = extractor.get_n_best(n=30)
        

        for val in keyphrases:
            out.append(val[0])
    except:
        out = []
        traceback.print_exc()

    return out

keywords = get_keywords(text)[:6]
print ("keywords: ",keywords)

keywords:  ['ibm watson', 'watson', 'machine learning', 'ibm', 'john mccarthy', 'natural language']


In [10]:
#TOkenize the text into sentences

sentences = tokenize_sentences(text)
print (sentences)

['“The science and engineering of making intelligent machines, especially intelligent computer programs”.', '-John McCarthy-\n\nJohn McCarthy\nArtificial Intelligence is an approach to make a computer, a robot, or a product to think how smart human think.', 'AI is a study of how human brain think, learn, decide and work, when it tries to solve problems.', 'And finally this study outputs intelligent software systems.The aim of AI is to improve computer functions which are related to human knowledge, for example, reasoning, learning, and problem-solving.', 'The intelligence is intangible.', 'It is composed of\nReasoning\nLearning\nProblem Solving\nPerception\nLinguistic Intelligence\nThe objectives of AI research are reasoning, knowledge representation, planning, learning, natural language processing, realization, and ability to move and manipulate objects.', 'There are long-term goals in the general intelligence sector.', 'Approaches include statistical methods, computational intelligen

In [11]:
#Extract sentences having the keywords that is extracted before.

from pprint import pprint
def get_sentences_for_keyword(keywords, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=False)
        keyword_sentences[key] = values
    return keyword_sentences

keyword_sentence_mapping = get_sentences_for_keyword(keywords, sentences)
pprint (keyword_sentence_mapping)

{'ibm': ['The supercomputer is named for IBM’s founder, Thomas J. Watson.',
         'Major Goals\n'
         'Knowledge reasoning\n'
         'Planning\n'
         'Machine Learning\n'
         'Natural Language Processing\n'
         'Computer Vision\n'
         'Robotics\n'
         'IBM Watson\n'
         '\n'
         '“Watson” is an IBM supercomputer that combines Artificial '
         'Intelligence (AI) and complex inquisitive programming for ideal '
         'execution as a “question answering” machine.',
         'At the point when IBM Watson made, IBM communicated that “more than '
         '100 particular techniques are used to inspect perceive sources, find '
         'and make theories, find and score affirm, and combination and rank '
         'speculations.” recently, the Watson limits have been expanded and '
         'the way by which Watson works has been changed to abuse new sending '
         'models (Watson on IBM Cloud) and propelled machine learning '
         'c

In [40]:
# len(keyword_sentence_mapping)
import re
answers = []
final_sentences = []
for k,v in keyword_sentence_mapping.items():
  if len(v)>0:
    match = v[0].lower()
    answers.append(k)
    if k in match:
      temp = re.compile(re.escape(k), re.IGNORECASE)
      final_sentences.append(temp.sub('______',match))
    else:
      final_sentences.append(match)
pprint(final_sentences)
pprint(answers)

['______ is at the forefront of the new era of computing.',
 'the supercomputer is named for ibm’s founder, thomas j. ______.',
 'cheat sheets for ai, neural networks, ______, deep learning & big data\n2.',
 'the supercomputer is named for ______’s founder, thomas j. watson.',
 '-______-\n'
 '\n'
 '______\n'
 'artificial intelligence is an approach to make a computer, a robot, or a '
 'product to think how smart human think.',
 '· ______ processing − interact with the computer that understands ______ '
 'spoken by humans.']
['ibm watson',
 'watson',
 'machine learning',
 'ibm',
 'john mccarthy',
 'natural language']


In [38]:
import random
from prettytable import PrettyTable
from IPython.display import Markdown, display

tab = PrettyTable()
random.shuffle(answers)
random.shuffle(final_sentences)
tab.field_names=['A', 'B']

def printmd(string):
    display(Markdown(string))
printmd('**Match column A with column B**')

for word,context in zip(answers,sent):
  tab.add_row([word,context])

print(tab)

**Match column A with column B**

+------------------+------------------------------------------------------------------------------------------------------------------+
|        A         |                                                        B                                                         |
+------------------+------------------------------------------------------------------------------------------------------------------+
|  john mccarthy   |                                                     -______-                                                     |
|                  |                                                                                                                  |
|                  |                                                      ______                                                      |
|                  | artificial intelligence is an approach to make a computer, a robot, or a product to think how smart human think. |
|       ibm        |                         the